In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, utils

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 40
BATCH_SIZE = 64

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("./data",
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("./data",
                          train=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE,
    shuffle=True
)

100.0%

Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw


111.0%

Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw


100.0%

Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw


159.1%E:\anaconda\envs\torch\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw
Processing...
Done!


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [4]:
def train(model, train_loader, optimizer):
    model.train()  # 모델을 학습 모드로 전환, 드롭아웃 등의 레이어에서 동작이 다름

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)  # 데이터도 처리할 디바이스로 보내야 함
        optimizer.zero_grad()  # 매 반복마다 기울기를 새로 계산해야 함
        output = model(data)
        loss = F.cross_entropy(output, target)  # 결과물과 정답 사이 크로스엔트로피 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 수정

        if batch_idx % 200 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}".format(epoch, batch_idx * len(data), len(train_loader.dataset), 100.*batch_idx/len(train_loader), loss.item()))

def evaluate(model, test_loader):
    model.eval()  # 모델을 평가 모드로 전환
    test_loss = 0
    correct = 0

    with torch.no_grad():  # 평가시엔 기울기 계산 필요없음
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction="sum").item()

            pred = output.max(1, keepdim=True)[1]  # 모델의 예측 클래스
            correct += pred.eq(target.view_as(pred)).sum().item()  # 정답이면 +1

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

In [5]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:2.302715
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.169809
Train Epoch: 1 [25600/60000 (43%)]	Loss:1.230328
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.871218
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.888999
[1] Test Loss: 0.6753, Accuracy: 73.16%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.795632
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.813564
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.715292
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.908739
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.647962
[2] Test Loss: 0.5739, Accuracy: 76.79%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.680826
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.503961
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.707063
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.639955
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.628808
[3] Test Loss: 0.5038, Accuracy: 80.45%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.549490
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.412589
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.495135
Train Epoch: 4 [38

[29] Test Loss: 0.3017, Accuracy: 88.97%
Train Epoch: 30 [0/60000 (0%)]	Loss:0.252678
Train Epoch: 30 [12800/60000 (21%)]	Loss:0.380046
Train Epoch: 30 [25600/60000 (43%)]	Loss:0.304281
Train Epoch: 30 [38400/60000 (64%)]	Loss:0.315912
Train Epoch: 30 [51200/60000 (85%)]	Loss:0.137905
[30] Test Loss: 0.3083, Accuracy: 88.85%
Train Epoch: 31 [0/60000 (0%)]	Loss:0.419524
Train Epoch: 31 [12800/60000 (21%)]	Loss:0.293344
Train Epoch: 31 [25600/60000 (43%)]	Loss:0.344983
Train Epoch: 31 [38400/60000 (64%)]	Loss:0.323253
Train Epoch: 31 [51200/60000 (85%)]	Loss:0.440610
[31] Test Loss: 0.2992, Accuracy: 89.37%
Train Epoch: 32 [0/60000 (0%)]	Loss:0.430285
Train Epoch: 32 [12800/60000 (21%)]	Loss:0.218076
Train Epoch: 32 [25600/60000 (43%)]	Loss:0.257397
Train Epoch: 32 [38400/60000 (64%)]	Loss:0.269211
Train Epoch: 32 [51200/60000 (85%)]	Loss:0.292565
[32] Test Loss: 0.3028, Accuracy: 89.16%
Train Epoch: 33 [0/60000 (0%)]	Loss:0.424127
Train Epoch: 33 [12800/60000 (21%)]	Loss:0.411549
Train 